## 需求描述
在游戏《幸福工厂》的生产体系中，配方是如$\sum_{i=1}^{n1} a_i \cdot x_i = \sum_{i=1}^{n1} b_i \cdot y_i$形式的公式，其中$x_i$是输入的原料，$a_i$是单位时间对应原料的消耗量，$y_i$是输出的产物，$b_i$是单位时间对应产物的产出量，一个配方的产出可能是另一个配方的输入，如此构成了生产链路，而链路交叉形成生产网络。
对于网络中的产物，可能存在多种配方，不同的配方有不同的原料消耗和生产效率，而基础原料的产量是有上限的，且丰度不同，于是便有了生产网络优化的需求。
对于配方生产网络，有m个item、n个formula两种类型节点，同种节点不互相连接。设节点的权重为$\alpha$和$\beta$，代表物资和配方的单位数量，边的权重为$W_{m\times n}$（代表每种配方每单位消耗产出物资数量）。item中有一些特殊的节点为源节点$s=\alpha_{m\times 1} \odot Ms_{m \times 1}$，其中$Ms_{m \times 1}$为标示源节点的掩码向量。源节点的需求不应该由formula产出，但是要累计计算资源需求。现需要对一些目标物资的生产路线建模，优化源节点的物资需求。
### 思路1 产出逆推需求
#### 推导过程
可以设想一种传播途径，从某些item节点开始，初始化权重为向量$\alpha^{pre}_{m\times 1}$，其中大于0的值表示需求，那么对于小于0表示已经提供的物资。对于权重为$\alpha_a>0$的节点$item_a$，需要通过生产公式消耗原料来满足其需求，假设其连接n个formula节点，权重为向量$w_{a|n}$，由于每个item可以由不同的formula输出，所以配置可训练向量$a_{a|n}$映射到概率向量$b_{a|n}=softmax(a_{a|n})$来表示每个item的产出需求由不同的formula输出提供的比例，满足$b_{ak}\cdot \alpha_a=ReLU(w_{ak})\cdot \beta_k$，即$\beta_k=\alpha_{ak} \cdot \frac{b_{ak}}{w_{ak}},k\in [1,n]$，同时由于每个formula有多个item产出，formula的实际需求权重应该由产出item中对formula需求最高的项决定，设$B_{m,n}=[b_1,b_2,...,b_m]$，有$\beta_k=max(\alpha \odot \frac{B_{:,k}}{W_{:,k}})$

那么推广到向量$\beta$，有
$$\begin{aligned}
&\alpha^{pre}_{m\times 1} \in R^{m\times 1} \quad &(当前的物资需求供应量)\\
&W_{m\times n} \in R^{m\times n} \quad &(生产权重矩阵)\\
&Wb_{m\times n}= \frac{1}{W_{m\times n}}\quad &(倒数生产权重矩阵，用于通过物资计算配方需求)\\
&A_{m\times n}\in R^{m\times n} \quad &(配方的分配权重矩阵，被训练参数)\\
&P_{m\times n}=softmax(A_{m\times n}, dim=1)\quad &(配方的分配概率矩阵)\\
&X_{m\times n}=ReLU(\alpha^{pre}_{m\times 1}) \odot P_{m\times n} \odot Wb_{m\times n}\quad &(物资对配方的需求强度矩阵)\\
&\beta_{n\times 1}= maxcol(X) \quad &(配方的需求矩阵)\\
\end{aligned}$$

但是会有一个问题，即$W_{m\times n}$的元素可能小于等于0，表示对应物资不是由对应配方生产，对应元素是无效的，不应该纳入计算，且要防止除0问题，所以应该引入掩码和epsilon，即
$$\begin{aligned}
&M_{m\times n} = \mathbb{I}(W_{m\times n}>0) \in \{0,1\}^{m\times n}\\
&Mp_{m\times n}=(1−M_{m\times n})\odot (-epsilon)\\
&Wb_{m\times n}= \frac{1}{W_{m\times n} + epsilon \odot \mathbb{I}(W_{m\times n}=0)}\\
&P_{m\times n}=softmax(A_{m\times n}+Mp_{m\times n}, dim=1) \odot M_{m\times n}\\
\end{aligned}$$
在实际运算中为了防止溢出，这里使用$-epsilon$代替$-\inf$，epsilon取一个极小数，比如1e-8。

以上网络进一步传播，由$\Delta\alpha_{m \times 1}=W_{m\times n} \cdot \beta_{n\times 1}$得到
$$\alpha'_{m\times 1}=\alpha^{pre}_{m\times 1} - \Delta\alpha_{m\times 1}=\alpha_{m\times 1}-W_{m\times n} \cdot \beta_{n\times 1}$$

另外，item中有一些特殊的节点为源节点$s=\alpha_{m\times 1} \odot Ms_{m \times 1}$，其中$Ms_{m \times 1}$为标示源节点的掩码向量。源节点的需求不应该由formula产出，但是要累计计算资源需求。即
$$s_{m \times 1}=s^{pre}_{m \times 1}+\alpha'_{m\times 1} \odot Ms_{m \times 1}$$
$$\alpha^{suf}_{m\times 1}=\alpha'_{m\times 1} \odot (1-Ms_{m \times 1})$$
#### 归纳建模
综上所述，已知边的权重矩阵为$W_{m\times n}$，标示源节点的掩码矩阵为$Ms_{m \times 1}\in \{0,1\}^{m\times 1}$，初始化需求供应矩阵为$\alpha^0_{m\times 1}$，建模方案如下：
$$\begin{aligned}
init:&\\
&W_{m\times n} \in R^{m\times n} \quad &(生产权重矩阵)\\
&Ms_{m \times 1}\in \{0,1\}^{m\times 1} \quad &(物资的源节点掩码矩阵)\\
&M_{m\times n}=\mathbb{I}(W_{m\times n}>0) \in \{0,1\}^{m\times n}\quad &(生产权重的供应掩码矩阵)\\\
&Ma_{m \times 1}=1-Ms_{m \times 1}\quad &(物资的非源节点掩码矩阵)\\
&Mp_{m\times n}=(1−M_{m\times n})\odot (-epsilon)\quad &(配方的分配概率掩码矩阵)\\
&Wb_{m\times n}= \frac{1}{W_{m\times n} + epsilon \odot \mathbb{I}(W_{m\times n}=0)}\quad &(倒数生产权重矩阵)\\
&\alpha^0_{m\times 1} \in R^{m\times 1} \quad &(初始需求供应矩阵)\\
&s^0_{m \times 1}=\alpha^0_{m\times 1} \odot Ms_{m \times 1}\quad &(初始的源节点累计需求矩阵)\\
......&\\
layer^i:&\\
&\alpha^{pre}_{m\times 1}\in R^{m\times 1} \quad &(由上一层的 \alpha^i_{m\times 1} 输入)\\
&s^{pre}_{m \times 1}\in R^{m\times 1} \quad &(由上一层的 s^i_{m \times 1} 输入)\\

&A^i_{m\times n}\in R^{m\times n} \quad &(配方的分配权重矩阵，被训练参数)\\
&P^i_{m\times n}=softmax(A^i_{m\times n}+Mp_{m\times n}, dim=1) \odot M_{m\times n} \quad &(配方的分配概率矩阵)\\
&X_{m\times n}=ReLU(\alpha^{pre}_{m\times 1}) \odot P^i_{m\times n} \odot Wb_{m\times n}\quad &(物资对配方的需求强度矩阵)\\
&\beta^i_{n\times 1}= maxcol(X) \quad &(配方的需求矩阵)\\
&\Delta\alpha_{m\times 1}=W_{m\times n} \cdot \beta^i_{n\times 1}\quad &(本层生产处理的的物资变化量)\\
&\alpha^i_{m\times 1}=\alpha^{pre}_{m\times 1} - \Delta\alpha_{m\times 1} \odot Ma_{m \times 1}\quad &(计算改变后的需求供应矩阵)\\
&s^i_{m \times 1}=s^{pre}_{m \times 1}+\Delta\alpha_{m\times 1} \odot Ms_{m \times 1}\quad &(提取累加源节点需求)\\
loss:&\\
&Loss=Loss_{source}(s^i_{m \times 1})+Loss_{unsat}(ReLU(−\alpha^i_{m\times 1}))
\end{aligned}$$

#### 复盘总结
针对以上建模，我使用多层网络去训练最优解，其中损失函数由最终&s^i_{m \times 1}$的基础资源的消耗损失和$ReLU(\alpha^i_{m\times 1})$的需求未满足损失，但是发现需求未满足损失无法下降。

总结原因是因为下游产物的需求满足往往会产生更多的上游产物的需求，所以此时在梯度上会趋向不满足下游产物的需求，从而导致需求未满足损失无法下降，另外在满足需求时，也会导致基础资源的消耗损失增加。

### 思路2 资源顺推产出
#### 推导过程

#### 归纳建模
已知边的权重矩阵为$W_{m\times n}$，目标产出的产物比例矩阵$Wa^0_{m\times 1} \in R_+^{m\times 1}$，初始的资源存量矩阵为$\alpha^0_{m \times 1}\in R_+^{m\times 1}$，建模方案如下：
$$\begin{aligned}
init:&\\
&\alpha^0_{m \times 1}\in R_+^{m\times 1} \quad &(初始资源存量矩阵)\\
&Wa_{m\times 1} \in R_+^{m\times 1} \quad &(目标产物的产物权重矩阵)\\

&W_{m\times n} \in R^{m\times n} \quad &(生产权重矩阵)\\
&M_{m\times n}=\mathbb{I}(W_{m\times n}<0) \in \{0,1\}^{m\times n}\quad &(生产权重的需求掩码矩阵)\\
&Mp_{m\times n}=(1−M_{m\times n})\odot (-epsilon)\quad &(配方的分配概率掩码矩阵)\\
&Wb_{m\times n}= \frac{1}{W_{m\times n} + epsilon \odot \mathbb{I}(W_{m\times n}=0)}\quad &(倒数生产权重矩阵)\\

......&\\
layer^i:&\\
&\alpha^{pre}_{m\times 1}\in R_+^{m\times 1} \quad &(由上一层的 \alpha^i_{m\times 1} 输入)\\
&A^i_{m\times n}\in R^{m\times n} \quad &(配方的分配权重矩阵，被训练参数)\\
&P^i_{m\times n}=softmax(A^i_{m\times n}+Mp_{m\times n}, dim=1) \odot M_{m\times n} \quad &(配方的分配概率矩阵)\\
&X_{m\times n}=-ReLU(\alpha^{pre}_{m\times 1}) \odot P^i_{m\times n} \odot Wb_{m\times n}\quad &(物资对配方的供应强度矩阵)\\
&\beta^i_{n\times 1}= mincol(X) \quad &(配方的供应矩阵)\\
&\Delta\alpha_{m\times 1}=W_{m\times n} \cdot \beta^i_{n\times 1}\quad &(本层生产处理的的物资变化量)\\
&\alpha^i_{m\times 1}=\alpha^{pre}_{m\times 1} + \Delta\alpha_{m\times 1}\quad &(计算改变后的资源存量矩阵)\\
loss:&\\
&Loss=Loss_{obj}(-\alpha^i_{m\times 1} \odot Wa_{m\times 1})
\end{aligned}$$

In [ ]:
import decimal
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import numpy as np
from typing import Callable
from factorio.satisfactory.formula import read_formula, parse_recipe_graph, parse_item, is_base_type, device


def print_item_weights(item_weights, idx2item: dict, cut=0.01):
    cut_str = str(cut).rstrip('0').rstrip('.') if '.' in str(cut) else str(cut)
    d = decimal.Decimal(cut_str)
    sign, digits, exponent = d.as_tuple()
    decimal_places = max(0, -int(exponent))
    
    # 步骤3：需要保留的位数 = 有效小数位数 + 1（或直接返回有效小数位数）
    needed_places = decimal_places + 1
    for idx in range(len(item_weights)):
        item_name = idx2item[idx]
        weight = np.round(item_weights[idx], needed_places)
        if np.abs(weight) > cut:
            print(f"  {item_name}: {weight}")

     
     
class RecipeGNN_V4(nn.Module):
    """
    配方生产网络：基于item-formula双节点类型的多层传播网络。
    item（资源存量）→formula（配方）→item（资源存量更新）
    """
    def __init__(self, G: nx.DiGraph, target_dict: dict, source_dict: dict,
                 num_layers: int=9,
                 epsilon=1e-8, device=device, dtype=torch.float32):
        super().__init__()
        self.device = device
        self.dtype = dtype
        self.G = G
        self.epsilon = epsilon
        
        self.parse_recipe_graph(weight = "speed")
          
        self.init_constants()
        self.init_target_weight(target_dict=target_dict)
        self.init_source_weight(source_dict=source_dict)
        # 可训练矩阵A^i（每层独立，按需扩展）
        self.init_layer(num_layers)
        # self.final_formula_weights = torch.zeros([len(self.formula_nodes)], dtype=dtype, device=device)
        # self.final_item_weights = torch.zeros([len(self.item_nodes)], dtype=dtype, device=device)

    
    def parse_recipe_graph(self, weight = "speed"):
        (
            self.raw_weight_matrix,
            self.item_nodes, self.formula_nodes,
            self.item2idx, self.formula2idx,
            self.idx2item, self.idx2formula
        ) = parse_recipe_graph(self.G, weight = weight)
        
    
    def init_layer(self, num_layers: int):
        """新增一层的可训练矩阵A^i"""
        self.A_layers = nn.ParameterList([
            nn.Parameter(torch.randn([len(self.item_nodes), len(self.formula_nodes)], dtype=self.dtype, device=self.device)) for _ in range(num_layers) # 初始化可训练参数，可自定义
        ])
    
    def init_target_weight(self, target_dict: dict):
        target_weight = torch.zeros(len(self.item_nodes), requires_grad=False, dtype=self.dtype, device=self.device)
        for key in target_dict:
            target_weight[self.item2idx[key]] = target_dict[key]
        self.target_weight = target_weight
        
    def init_source_weight(self, source_dict: dict = {}):
        source_weight = torch.zeros(len(self.item_nodes), requires_grad=False, dtype=self.dtype, device=self.device)
        for key in source_dict:
            source_weight[self.item2idx[key]] = source_dict[key]
        self.alpha_0 = source_weight
        
    def init_constants(self):
        """
        初始化所有常量（仅需调用一次）
        """
        device = self.device
        dtype = self.dtype
        epsilon = self.epsilon
        
        self.fixed_weight_matrix = torch.tensor(self.raw_weight_matrix, requires_grad=False, dtype=dtype, device=device) 
        # 边掩码矩阵 M = I(W<0) ，获取 item -> formula 的有效边
        self.weight_mask = (self.fixed_weight_matrix < 0).float()
        # 分配权重的掩码权重矩阵 Mp = (1-M) * (-inf) 
        # 将 item -> formula 的无效边相关的矩阵设置为 -inf
        self.invalid_mask_weight = torch.where(self.weight_mask.bool(), 0, -torch.inf)
        # 3. 倒数权重矩阵 Wb = 1 / max(W, epsilon) ，计算 item 的 formula 需求
        self.reciprocal_weight_matrix = 1.0 / (self.fixed_weight_matrix + epsilon * (self.fixed_weight_matrix == 0).float())
        
    
    def single_layer_forward(self, alpha_prev: torch.Tensor, layer_idx):
        """
        单轮layer_i前向计算（仅处理可变部分）
            参数：
                alpha_prev: (m,1) 上一轮item权重 α^{i-1}
                layer_idx: int 当前层索引（从0开始）
            返回：
                alpha_i: (m,1) 非源节点item权重 α^i
                s_increment: (m,1) 源节点资源增量（用于累计）
                beta_i: (n,1) 当前层配方权重
                P_i: 当前层配方分配概率矩阵
        """
        alpha_prev = alpha_prev.unsqueeze(1)
        # 1. 获取当前层可训练矩阵A^i
        A_i = self.A_layers[layer_idx]
        
        # 2. 计算P^i = M ⊙ softmax(A^i + Mp, dim=1)
        P_i = self.weight_mask * F.softmax(A_i + self.invalid_mask_weight, dim=1).nan_to_num(nan=0)
        # 3. 计算beta^i = maxcol(α_prev ⊙ P_i ⊙ Wb)
        X = F.relu(alpha_prev) * P_i * self.reciprocal_weight_matrix + self.invalid_mask_weight
        X = torch.nan_to_num(X, nan=-torch.inf, neginf=-torch.inf)
        beta_i = X.max(dim=0, keepdim=True)[0].T  # maxcol
        beta_i = -torch.nan_to_num(beta_i, neginf=0)
        # 4. 计算alpha' = ReLU(α_prev - W·β^i)
        delta_alpha = self.fixed_weight_matrix @ beta_i  # W是init_constants中保存的原始矩阵
        alpha_i = alpha_prev + delta_alpha
        
        return alpha_i.squeeze(1), beta_i, X, P_i
    
    def forward(self, alpha_0: torch.Tensor):
        """
        完整前向传播（含初始化+多轮计算）
        参数：
            alpha_0: (m,1) 初始资源存量
        返回：
            resource_need: alpha_prev + s_prev
            layer_list["alpha": alpha_i, "s": s_i, "beta": beta_i, "P": P_i]
        """
        
        # 4. 多轮传播迭代
        num_layers = len(self.A_layers)
        alpha_prev = alpha_0
        layer_list = []
        beta_list = []
        for i in range(num_layers):
            # 单轮层计算（仅处理可变参数）
            alpha_i, beta_i, contribution, P_i = self.single_layer_forward(
                alpha_prev, layer_idx=i
            )
            beta_list.append(beta_i)
            
            # 保存结果
            layer_list.append({
                "alpha": alpha_i, "beta": beta_i, "X": contribution, "P": P_i
            })

            # 更新迭代变量
            alpha_prev = alpha_i
            
        self.final_formula_weights = torch.cat(beta_list, dim=1).sum(dim=1)
        self.final_item_weights = (self.fixed_weight_matrix @ self.final_formula_weights.unsqueeze(1)).squeeze(1)
        self.resource, self.layer_list = alpha_prev, layer_list
        return self.resource, layer_list

    def print_final_formula_weights(self, cut=0.001):
        final_formula_weights = self.final_formula_weights.cpu().detach().numpy()
        print("\n最终配方权重：")
        print_item_weights(final_formula_weights, self.idx2formula, cut)
    

        
    def print_final_item_weights(self, cut=0.01):
        final_item_weights = self.final_item_weights.cpu().detach().numpy()
        print("\n最终物料权重：")
        print_item_weights(final_item_weights, self.idx2item, cut)

def recipe_loss_fn(
    final_item_weights: torch.Tensor,
    target_weights: torch.Tensor
) -> tuple[torch.Tensor, dict]:
    """
    多目标损失函数设计
    参数：
        lambda_target: 目标物料的损失权重（优先级最高，指数递增）
        lambda_source_max: 基础资源最大化的损失权重（线性递增）
        lambda_other_pos: 其他资源大于0的损失权重（线性递增）
        target_num: 目标物料的数量
    返回：
        total_loss: 总损失（可反向传播优化）
        dict: loss_source_max loss_other_pos
    """
    
    loss_target_max = -torch.sum(final_item_weights * target_weights)
    total_loss = (
        loss_target_max
    )

    # 返回总损失及各分项损失（方便监控）
    return total_loss, {
        'loss_source_max': loss_target_max.cpu().item()
    }
    
def train_recipe_gnn(model: RecipeGNN_V4,
        source_dict: dict | None = None,
        target_dict: dict | None = None,
        epochs: int = 1000,
        lr: float = 0.01,
        print_interval: int = 100,
    ) -> None:
        """
        训练配方GNN模型
        参数：
            target_base_ratio: 基础资源的目标配比（numpy数组，长度=基础资源数量）
            epochs: 训练轮数
            lr: 学习率
            print_interval: 日志打印间隔
            """
        # 1. 配置优化器（仅优化可训练参数 formula_weights）
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        if source_dict is not None:
            model.init_source_weight(source_dict)
        if target_dict is not None:
            model.init_target_weight(target_dict)
        # 3. 训练循环
        model.train()  # 切换到训练模式
        for epoch in range(epochs):
            # 梯度清零
            optimizer.zero_grad()

            # 前向传播：获取资源增减量
            resource, layer_list = model.forward(alpha_0=model.alpha_0)

            # 计算损失
            total_loss, loss_details = recipe_loss_fn(
                model.resource,
                target_weights=model.target_weight
            )

            model.total_loss = total_loss
            # 反向传播 + 参数更新
            total_loss.backward()
            optimizer.step()

            # 打印训练日志
            if (epoch + 1) % print_interval == 0:
                print(f"Epoch [{epoch+1}/{epochs}]")
                print(f"  总损失：{total_loss.item():.4f}")
                # 打印关键指标
                target_change = model.final_item_weights[(model.target_weight > 0)]
                target_change = target_change.cpu().detach().numpy()
                source_changes = model.final_item_weights[(model.alpha_0 > 0)]
                source_changes = source_changes.cpu().detach().numpy()
                
                # 避免除0错误
                if np.sum(np.abs(source_changes)) > 1e-8:
                    base_ratio = source_changes / np.sum(np.abs(source_changes))
                else:
                    base_ratio = source_changes
                print(f"  目标物料增减量：{target_change.round(4)}")
                print(f"  基础资源增减量：{source_changes.round(2)}")
                print(f"  基础资源当前配比：{base_ratio.round(4)}")
                print("-" * 50)

        print("训练完成！")
        model.print_final_formula_weights(cut=0.001)
        model.print_final_item_weights(cut=0.01)

if __name__ == "__main__":
    g = read_formula()

    num_layers = 20  # 传播层数
    epsilon = 1e-8  # 数值安全常数
    
    source_weight={
        "水": 1e8, "激发态光子物质": 1e8, "铁矿石": 921, "石灰石": 693, "铜矿石": 369, "钦金矿石": 150, "煤": 423,
        "原油": 126, "硫磺": 108, "铝土矿": 123, "粗石英": 135, "铀": 21, "SAM物质": 102, "氮气": 120
    }
    
    # 3. 初始化模型

    source_weight = {k:v * 1e3 for k,v in source_weight.items()}
    target_weight={
        "封压方块": 1,
        "镄燃料棒": 2
    }
    model = RecipeGNN_V4(g, target_dict=target_weight, source_dict=source_weight, num_layers=num_layers, epsilon=epsilon)
    
    train_recipe_gnn(
        model=model,
        source_dict=source_weight,
        target_dict=target_weight,
        epochs=1000,
        lr=0.01,
        print_interval=100
    )
    
    # 网络传递的系统误差
    sys_loss = torch.sqrt(torch.nn.functional.mse_loss(model.final_item_weights + model.alpha_0,  model.resource))/ torch.mean(model.resource)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    optimizer.zero_grad()

    resource, layer_list = model.forward(alpha_0=model.alpha_0)

    total_loss, loss_details = recipe_loss_fn(
        resource,
        target_weights=model.target_weight,
    )

    torch.autograd.grad(outputs=total_loss, retain_graph=True, inputs=model.layer_list[-1]["beta"])

当前使用设备：cuda:0


/home/bio-24/projects/local/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import decimal
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import numpy as np
from typing import Callable
from factorio.satisfactory.formula import read_formula, parse_recipe_graph, parse_item, is_base_type, device


def print_item_weights(item_weights, idx2item: dict, cut=0.01):
    cut_str = str(cut).rstrip('0').rstrip('.') if '.' in str(cut) else str(cut)
    d = decimal.Decimal(cut_str)
    sign, digits, exponent = d.as_tuple()
    decimal_places = max(0, -int(exponent))
    
    # 步骤3：需要保留的位数 = 有效小数位数 + 1（或直接返回有效小数位数）
    needed_places = decimal_places + 1
    for idx in range(len(item_weights)):
        item_name = idx2item[idx]
        weight = np.round(item_weights[idx], needed_places)
        if np.abs(weight) > cut:
            print(f"  {item_name}: {weight}")

           
class RecipeGNN(nn.Module):
    """
    配方生产网络：基于item-formula双节点类型的多层传播网络。
    item（物资）→formula（配方）→item更新→源节点资源累计
    """
    def __init__(self, G: nx.DiGraph, target_dict: dict, source_weight: dict,
                 num_layers: int=9,
                 epsilon=1e-8, device=device, dtype=torch.float32,
                 is_source: Callable[[nx.DiGraph, str], bool] = is_base_type):
        super().__init__()
        self.device = device
        self.dtype = dtype
        self.G = G
        self.epsilon = epsilon
        
        self.parse_recipe_graph(weight = "speed")
        
        self.parse_item(target_dict=target_dict, is_source=is_source)
        
        self.init_constants()
        self.init_source_weight(source_weight=source_weight)
        # 可训练矩阵A^i（每层独立，按需扩展）
        self.init_layer(num_layers)
        # self.final_formula_weights = torch.zeros([len(self.formula_nodes)], dtype=dtype, device=device)
        # self.final_item_weights = torch.zeros([len(self.item_nodes)], dtype=dtype, device=device)

    
    def parse_recipe_graph(self, weight = "speed"):
        (
            self.raw_weight_matrix,
            self.item_nodes, self.formula_nodes,
            self.item2idx, self.formula2idx,
            self.idx2item, self.idx2formula
        ) = parse_recipe_graph(self.G, weight = weight)
        
    
    def parse_item(self, target_dict: dict, is_source = is_base_type):
        (
            self.raw_target_weights, self.source_items, self.source_idxs,
            self.target_items, self.target_idxs, self.other_items, self.other_items_idx
        ) = parse_item(self.G, item_nodes=self.item_nodes, item2idx=self.item2idx,
                       target_dict=target_dict, is_source=is_source)
    
    def init_layer(self, num_layers: int):
        """新增一层的可训练矩阵A^i"""
        self.A_layers = nn.ParameterList([
            nn.Parameter(torch.randn([len(self.item_nodes), len(self.formula_nodes)], dtype=self.dtype, device=self.device)) for _ in range(num_layers) # 初始化可训练参数，可自定义
        ])
        
    def init_source_weight(self, source_weight: dict = {}, default = 1):
        min_val, max_val = 0, 1
        target_source_weight = torch.zeros(len(self.item_nodes), requires_grad=False, dtype=self.dtype, device=self.device)
        for key in self.source_items:
            if key in source_weight:
                target_source_weight[self.item2idx[key]] = source_weight[key]
            else:
                target_source_weight[self.item2idx[key]] = default
        # 计算当前权重的最小值和最大值（添加epsilon避免除0）
        weight_min = torch.clamp(torch.min(target_source_weight), min=self.epsilon)
        weight_max = torch.clamp(torch.max(target_source_weight), min=self.epsilon)
        # 最小-最大缩放：将权重缩放到 [min_val, max_val]
        scaled_weight = (target_source_weight - weight_min) / (weight_max - weight_min) * (max_val - min_val) + min_val
        self.source_weight = source_weight
        self.scaled_source_weight = scaled_weight
        
    def init_constants(self):
        """
        初始化所有常量（仅需调用一次）
        """
        device = self.device
        dtype = self.dtype
        epsilon = self.epsilon
        
        self.fixed_weight_matrix = torch.tensor(self.raw_weight_matrix, requires_grad=False, dtype=dtype, device=device) 
        self.source_mask = torch.zeros([len(self.item_nodes)], requires_grad=False, dtype=dtype, device=device)
        self.source_mask[self.source_idxs] = 1.0
        self.target_weights = torch.tensor(self.raw_target_weights, requires_grad=False, dtype=dtype, device=device)
        # 1. 边掩码矩阵 M = I(W>0) ，获取 item -> formula 的有效边
        self.weight_mask = (self.fixed_weight_matrix > 0).float()
        
        # 2. 概率掩码矩阵 Mp = (1-M) * (-1/epsilon) ，获取 item -> formula 的有效概率选择
        minus_1_over_epsilon = -1.0 / epsilon
        self.probability_mask = (1 - self.weight_mask) * minus_1_over_epsilon
        
        # 3. 倒数权重矩阵 Wb = 1 / max(W, epsilon) ，计算 item 的 formula 需求
        self.reciprocal_weight_matrix = 1.0 / torch.max(self.fixed_weight_matrix, torch.tensor(epsilon, requires_grad=False, dtype=dtype, device=device))
        
        # 4. 非源节点掩码 Ma = 1 - Ms
        self.normal_mask = (1 - self.source_mask).float()
        
        self.alpha_0 = torch.tensor(self.raw_target_weights, requires_grad=False, dtype=dtype, device=device)
        self.s_0 = torch.zeros([len(self.item_nodes)], requires_grad=False, dtype=dtype, device=device)
        self.s_0 = self.s_0 + self.alpha_0 * self.source_mask
        
    
    def single_layer_forward(self, alpha_prev: torch.Tensor, layer_idx):
        """
        单轮layer_i前向计算（仅处理可变部分）
            参数：
                alpha_prev: (m,1) 上一轮item权重 α^{i-1}
                layer_idx: int 当前层索引（从0开始）
            返回：
                alpha_i: (m,1) 非源节点item权重 α^i
                s_increment: (m,1) 源节点资源增量（用于累计）
                beta_i: (n,1) 当前层配方权重
                P_i: 当前层配方分配概率矩阵
        """
        alpha_prev = alpha_prev.unsqueeze(1)
        # 1. 获取当前层可训练矩阵A^i
        A_i = self.A_layers[layer_idx]
        
        # 2. 计算P^i = M ⊙ softmax(A^i + Mp, dim=1)
        P_i = self.weight_mask * F.softmax(A_i + self.probability_mask, dim=1)
        
        # 3. 计算beta^i = maxcol(α_prev ⊙ P_i ⊙ Wb)
        contribution = F.relu(alpha_prev) * P_i * self.reciprocal_weight_matrix  # 哈达玛积简化
        
        beta_i: torch.Tensor = contribution.max(dim=0, keepdim=True)[0].T  # maxcol
        # 4. 计算alpha' = ReLU(α_prev - W·β^i)
        delta_alpha = self.fixed_weight_matrix @ beta_i  # W是init_constants中保存的原始矩阵
        alpha_prime = (alpha_prev - delta_alpha).squeeze(1)
        
        # 5. 源节点资源增量 + 非源节点alpha_i
        s_increment = alpha_prime * self.source_mask
        alpha_i = alpha_prime * self.normal_mask
        
        return alpha_i, s_increment, beta_i, P_i
    
    def forward(self, alpha_0: torch.Tensor, s_0: torch.Tensor):
        """
        完整前向传播（含初始化+多轮计算）
        参数：
            alpha_0: (m,1) 初始item需求
            s_0: (m,1) 初始源节点资源需求
        返回：
            resource_need: alpha_prev + s_prev
            layer_list["alpha": alpha_i, "s": s_i, "beta": beta_i, "P": P_i]
        """
        
        # 4. 多轮传播迭代
        num_layers = len(self.A_layers)
        alpha_prev = alpha_0
        s_prev = s_0
        layer_list = []
        beta_list = []
        for i in range(num_layers):
            # 单轮层计算（仅处理可变参数）
            alpha_i, s_inc, beta_i, P_i = self.single_layer_forward(
                alpha_prev, layer_idx=i
            )
            beta_list.append(beta_i)
            # 累计源节点资源 s^i = s^{i-1} + s_inc
            s_i = s_prev + s_inc
            
            # 保存结果
            layer_list.append({
                "alpha": alpha_i, "s": s_i, "beta": beta_i, "P": P_i
            })

            # 更新迭代变量
            alpha_prev = alpha_i
            s_prev = s_i
        self.final_formula_weights = torch.cat(beta_list, dim=1).sum(dim=1)
        self.final_item_weights = (self.fixed_weight_matrix @ self.final_formula_weights.unsqueeze(1)).squeeze(1)
        self.resource_need, self.layer_list = alpha_prev + s_prev, layer_list
        self.layer_list = layer_list
        return self.resource_need, layer_list

    def print_final_formula_weights(self, cut=0.001):
        final_formula_weights = self.final_formula_weights.cpu().detach().numpy()
        print("\n最终配方权重：")
        print_item_weights(final_formula_weights, self.idx2formula, cut)
    

        
    def print_final_item_weights(self, cut=0.01):
        final_item_weights = self.final_item_weights.cpu().detach().numpy()
        print("\n最终物料权重：")
        print_item_weights(final_item_weights, self.idx2item, cut)
    
def recipe_loss_fn(
    final_item_weights: torch.Tensor,
    scaled_source_weight: torch.Tensor,
    other_items_idx,
    lambda_source_max: float = 0.1,     # 基础资源最大化的损失权重
    lambda_other_pos: float = 2.0,     # 其他资源大于0的损失权重
    epsilon=1e-8,
) -> tuple[torch.Tensor, dict]:
    """
    多目标损失函数设计
    参数：
        lambda_target: 目标物料的损失权重（优先级最高，指数递增）
        lambda_source_max: 基础资源最大化的损失权重（线性递增）
        lambda_other_pos: 其他资源大于0的损失权重（线性递增）
        target_num: 目标物料的数量
    返回：
        total_loss: 总损失（可反向传播优化）
        dict: loss_source_max loss_other_pos
    """
    # # -------------------------- 损失项1：基础资源尽可能消耗少 --------------------------
    # # 提取基础资源的增减量 [len(source_resources),]
    # source_change = final_item_weights * scaled_source_weight
    # # 目标：因为source_change为负值表示消耗，所以应该最大化 source_change → 损失函数中最小化 (-source_change) 的均值（越大，损失越小）
    # source_change_norm = torch.nn.functional.normalize(source_change, p=2, dim=0, eps=epsilon)
    # # 步骤2：用归一化后的数值计算损失，梯度幅值将固定
    # loss_source_max = -torch.sum(source_change_norm * scaled_source_weight)
    
    loss_source_max = -torch.sum(final_item_weights * scaled_source_weight)
    # -------------------------- 损失项2：其他资源大于0（非基础、非目标） --------------------------
    # 提取其他资源的索引：非基础 + 非目标

    other_change = final_item_weights[other_items_idx]
    
    # 惩罚小于0的部分：使用ReLU的反向逻辑（若x<0，惩罚为-x；x≥0，惩罚为0）
    loss_other_pos = torch.max(torch.nn.functional.relu(-other_change))  # other_change<0时，-other_change>0，产生惩罚

    # -------------------------- 总损失：加权组合 --------------------------
    total_loss = (
        lambda_source_max * loss_source_max
        + lambda_other_pos * loss_other_pos
    )

    # 返回总损失及各分项损失（方便监控）
    return total_loss, {
        'loss_source_max': loss_source_max.cpu().item(),
        'loss_other_pos': loss_other_pos.cpu().item()
    }
    
def train_recipe_gnn(model: RecipeGNN,
        source_weight: dict | None = None,
        epochs: int = 1000,
        lr: float = 0.01,
        print_interval: int = 100,
        lambda_source_max = 0.1,     # 基础资源最大化的损失权重
        lambda_other_pos = 10.0,     # 其他资源大于0的损失权重
    ) -> None:
        """
        训练配方GNN模型
        参数：
            target_base_ratio: 基础资源的目标配比（numpy数组，长度=基础资源数量）
            epochs: 训练轮数
            lr: 学习率
            print_interval: 日志打印间隔
            """
        # 1. 配置优化器（仅优化可训练参数 formula_weights）
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        if source_weight is not None:
            model.init_source_weight(source_weight)
        # 3. 训练循环
        model.train()  # 切换到训练模式
        for epoch in range(epochs):
            # 梯度清零
            optimizer.zero_grad()

            # 前向传播：获取资源增减量
            resource_need, layer_list = model.forward(alpha_0=model.alpha_0, s_0=model.s_0)

            # 计算损失
            total_loss, loss_details = recipe_loss_fn(
                model.final_item_weights,
                scaled_source_weight=model.scaled_source_weight,
                other_items_idx=model.other_items_idx,
                lambda_source_max = lambda_source_max,     # 基础资源最大化的损失权重
                lambda_other_pos = lambda_other_pos,     # 其他资源大于0的损失权重
            )

            model.total_loss = total_loss
            # 反向传播 + 参数更新
            total_loss.backward()
            optimizer.step()

            # 打印训练日志
            if (epoch + 1) % print_interval == 0:
                print(f"Epoch [{epoch+1}/{epochs}]")
                print(f"  总损失：{total_loss.item():.4f}")
                print(f"  基础资源最大化损失：{loss_details['loss_source_max']:.4f}")
                print(f"  其他资源非负损失：{loss_details['loss_other_pos']:.4f}")
                # 打印关键指标
                target_change = model.final_item_weights[model.target_idxs].cpu().detach().numpy()
                source_changes = resource_need[model.source_idxs].cpu().detach().numpy()
                
                # 避免除0错误
                if np.sum(np.abs(source_changes)) > 1e-8:
                    base_ratio = source_changes / np.sum(np.abs(source_changes))
                else:
                    base_ratio = source_changes
                print(f"  目标物料增减量：{target_change.round(4)}")
                print(f"  基础资源增减量：{source_changes.round(2)}")
                print(f"  基础资源当前配比：{base_ratio.round(4)}")
                print("-" * 50)

        print("训练完成！")
        model.print_final_formula_weights(cut=0.001)
        model.print_final_item_weights(cut=0.01)

# -------------------------- 测试示例 --------------------------
if __name__ == "__main__":
    g = read_formula()

    num_layers = 20  # 传播层数
    epsilon = 1e-8  # 数值安全常数
    
    source_weight={
        "水": 0, "激发态光子物质": 0, "铁矿石": 1/921, "石灰石": 1/693, "铜矿石": 1/369, "钦金矿石": 1/150, "煤": 1/423,
        "原油": 1/126, "硫磺": 1/108, "铝土矿": 1/123, "粗石英": 1/135, "铀": 1/21, "SAM物质": 1/102, "氮气": 1/120
    }
    target_weight={
        "封压方块": 1,
        "镄燃料棒": 2
    }
    
    # 3. 初始化模型

    
    model = RecipeGNN(g, target_dict=target_weight, source_weight=source_weight, num_layers=num_layers, epsilon=epsilon)
    
    train_recipe_gnn(
        model=model,
        source_weight=source_weight,
        epochs=1000,
        lr=0.01,
        print_interval=100,
        lambda_source_max=0.01,
        lambda_other_pos=100
    )
    
    # 网络传递的系统误差
    sys_loss = torch.sqrt(torch.nn.functional.mse_loss(model.final_item_weights,  -model.resource_need))/ torch.mean(model.resource_need)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    optimizer.zero_grad()

    resource_need, layer_list = model.forward(alpha_0=model.alpha_0, s_0=model.s_0)

    total_loss, loss_details = recipe_loss_fn(
        -resource_need,
        scaled_source_weight=model.scaled_source_weight,
        other_items_idx=model.other_items_idx,
        lambda_source_max = 1,     # 基础资源最大化的损失权重
        lambda_other_pos = 1,     # 其他资源大于0的损失权重
    )

    torch.autograd.grad(outputs=total_loss, retain_graph=True, inputs=model.layer_list[-1]["beta"])

RuntimeError: The size of tensor a (155) must match the size of tensor b (228) at non-singleton dimension 2